## Data Scrubbing

In [1]:
import numpy as np
import pandas as pd

input_file = "./train.csv"

def AgeGroup(x):
    if x < 13:
        return 0
    elif x < 18:
        return 1
    elif x < 60:
        return 2
    else:
        return 3

def Embark(x):
    if x == 'S':
        return 0
    elif x == 'C':
        return 1
    elif x == 'Q':
        return 2
    else:
        return x
    
def FamilySize(x):
    if x == 1:
        return 0
    elif x < 5:
        return 1
    else:
        return 2
    
def Title(x):
    if x == 'Mlle':
        return 'Miss'
    elif x == 'Ms':
        return 'Miss'
    elif x == 'Mme':
        return 'Mrs'
    elif x == 'Lady':
        return 'Miss'
    elif x  == 'Dona':
        return 'Miss'
    elif x  == 'Capt':
        return 'Officer'
    elif x == 'Col':
        return 'Officer'
    elif x == 'Major':
        return 'Officer'
    elif x == 'Dr':
        return 'Officer'
    elif x == 'Rev':
        return 'Officer'
    elif x == 'Don':
        return 'Officer'
    elif x == 'Sir':
        return 'Officer'
    elif x == 'the Countess':
        return 'Officer'
    elif x == 'Jonkheer':
        return 'Officer'
    else:
        return x
    
def TitleGroup(x):
    if x == 'Mr':
        return 0
    elif x == 'Miss':
        return 1
    elif x == 'Mrs':
        return 2
    elif x == 'Master':
        return 3
    elif x == 'Officer':
        return 4
    else:
        return 5
    
def TicketSize(x):
    if x == 1:
        return 0
    elif x < 5:
        return 1
    else:
        return 2
        

# comma delimited is the default
passengers = pd.read_csv(input_file, header = 0)

passengers['Sex'] = passengers \
    .apply(lambda row: row['Sex'] == 'female', axis = 1).astype(int)

passengers['Age'] = passengers['Age'].fillna(round(passengers['Age'].mean(), 2))

passengers['AgeGroup'] = passengers['Age'].apply(lambda x: AgeGroup(x))

passengers['Embarked'] = passengers['Embarked'] \
    .fillna(passengers['Embarked'].value_counts().idxmax())
    
passengers['Embarked'] = passengers['Embarked'] \
    .apply(lambda x: Embark(x)).astype(int)

passengers['Title'] = passengers['Name'] \
    .replace("^.*, (.*?)\\..*$", "\\1", regex = True) \
    .apply(lambda x: Title(x))

passengers['FamilySize'] = passengers[['SibSp', 'Parch']] \
    .apply(lambda x: FamilySize(x['SibSp'] + x['Parch'] + 1), axis = 1)
    
passengers['TitleGroup'] = passengers['Title'].apply(lambda x: TitleGroup(x))

passengers['TicketSize'] = passengers.groupby(['Ticket'])['Ticket'].transform('count')
passengers['TicketSize'] = passengers['TicketSize'].apply(lambda x: TicketSize(x))

pd.DataFrame.head(passengers, 5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeGroup,Title,FamilySize,TitleGroup,TicketSize
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,Mr,1,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,Mrs,1,2,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,2,Miss,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,Mrs,1,2,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,2,Mr,0,0,0


## Dicision Tree Setup

In [2]:
from sklearn.tree import DecisionTreeClassifier

X = passengers[['Pclass', 'Sex', 'Embarked', 'AgeGroup', 'FamilySize', 'TitleGroup']]
y = passengers['Survived']

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## Create Decision Tree Graph

In [3]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file = 'survivor.dot',
    feature_names = ['Pclass', 'Sex', 'Embarked', 'AgeGroup', 'FamilySize', 'TitleGroup'],
    class_names = ['Perished', 'Survived'],
    rounded = True,
    filled = True
)

In [4]:
from io import StringIO
from sklearn import tree

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

#from graphviz import *

import matplotlib.image as mpimg
import pydotplus

classifier = tree_clf
dotfile = open("survivor.dot", 'w')
tree.export_graphviz(
    classifier,
    out_file = dotfile,
    feature_names = ['Pclass', 'Sex', 'Embarked', 'AgeGroup', 'FamilySize', 'TitleGroup'],
    class_names = ['Perished', 'Survived'],
    rounded = True,
    filled = True
)
dotfile.close()
dot_data = StringIO()
tree.export_graphviz(
    classifier,
    out_file = dot_data,
    feature_names = ['Pclass', 'Sex', 'Embarked', 'AgeGroup', 'FamilySize', 'TitleGroup'],
    class_names = ['Perished', 'Survived'],
    rounded = True,
    filled = True
)
graph = pydotplus.graph_from_dot_file("survivor.dot")
graph.write_png('survivor.png')
img = mpimg.imread('survivor.png')
fig = plt.figure(figsize=(100, 100)) 
plt.axis("off")
plt.imshow(img, cmap = plt.cm.binary, interpolation="nearest")
plt.show()

InvocationException: GraphViz's executables not found

## Test Data Scrubbing

In [ ]:
input_test_file = "./test.csv"

def AgeGroup(x):
    if x < 13:
        return 0
    elif x < 18:
        return 1
    elif x < 60:
        return 2
    else:
        return 3

def Embark(x):
    if x == 'S':
        return 0
    elif x == 'C':
        return 1
    elif x == 'Q':
        return 2
    else:
        return x
    
def FamilySize(x):
    if x == 1:
        return 0
    elif x < 5:
        return 1
    else:
        return 2
    
def Title(x):
    if x == 'Mlle':
        return 'Miss'
    elif x == 'Ms':
        return 'Miss'
    elif x == 'Mme':
        return 'Mrs'
    elif x == 'Lady':
        return 'Miss'
    elif x  == 'Dona':
        return 'Miss'
    elif x  == 'Capt':
        return 'Officer'
    elif x == 'Col':
        return 'Officer'
    elif x == 'Major':
        return 'Officer'
    elif x == 'Dr':
        return 'Officer'
    elif x == 'Rev':
        return 'Officer'
    elif x == 'Don':
        return 'Officer'
    elif x == 'Sir':
        return 'Officer'
    elif x == 'the Countess':
        return 'Officer'
    elif x == 'Jonkheer':
        return 'Officer'
    else:
        return x
    
def TitleGroup(x):
    if x == 'Mr':
        return 0
    elif x == 'Miss':
        return 1
    elif x == 'Mrs':
        return 2
    elif x == 'Master':
        return 3
    elif x == 'Officer':
        return 4
    else:
        return 5
        
def TicketSize(x):
    if x == 1:
        return 0
    elif x < 5:
        return 1
    else:
        return 2
        
# comma delimited is the default
test_passengers = pd.read_csv(input_test_file, header = 0)

test_passengers['Sex'] = test_passengers \
    .apply(lambda row: row['Sex'] == 'female', axis = 1).astype(int)

test_passengers['Age'] = test_passengers['Age'] \
    .fillna(round(test_passengers['Age'].mean(), 2))

test_passengers['AgeGroup'] = test_passengers['Age'].apply(lambda x: AgeGroup(x))

test_passengers['Embarked'] = test_passengers['Embarked'] \
    .fillna(test_passengers['Embarked'].value_counts().idxmax())
    
test_passengers['Embarked'] = test_passengers['Embarked'] \
    .apply(lambda x: Embark(x)).astype(int)

test_passengers['Title'] = test_passengers['Name'] \
    .replace("^.*, (.*?)\\..*$", "\\1", regex = True) \
    .apply(lambda x: Title(x))

test_passengers['FamilySize'] = test_passengers[['SibSp', 'Parch']] \
    .apply(lambda x: FamilySize(x['SibSp'] + x['Parch'] + 1), axis = 1)
    
test_passengers['TitleGroup'] = test_passengers['Title'].apply(lambda x: TitleGroup(x))

test_passengers['TicketSize'] = test_passengers \
    .groupby(['Ticket'])['Ticket'].transform('count')
test_passengers['TicketSize'] = test_passengers['TicketSize'].apply(lambda x: TicketSize(x))

pd.DataFrame.head(test_passengers, 5)

## Predict Probability of Survival for Test Data

In [ ]:
tree_clf.predict_proba(test_passengers[['Pclass', 'Sex', 'Embarked', 'AgeGroup', 'FamilySize', 'TitleGroup']])[:10]

In [ ]:
test_passengers['Survived'] = tree_clf \
    .predict(test_passengers[['Pclass', 'Sex', 'Embarked', 'AgeGroup', 'FamilySize', 'TitleGroup']])

## Export Prediction to CSV File

In [ ]:
test_passengers[['PassengerId', 'Survived']] \
    .to_csv('test_(Pclass_Sex_Embarked_AgeGroup_FamilySize_TitleGroup)_results.csv', sep=',', index = False, encoding='utf-8')
pd.DataFrame.head(test_passengers)

## Data Scrubbing

In [ ]:
import numpy as np
import pandas as pd

input_file = "./train.csv"

def AgeGroup(x):
    if x < 13:
        return 0
    elif x < 18:
        return 1
    elif x < 60:
        return 2
    else:
        return 3

def Embark(x):
    if x == 'S':
        return 0
    elif x == 'C':
        return 1
    elif x == 'Q':
        return 2
    else:
        return x
    
def FamilySize(x):
    if x == 1:
        return 0
    elif x < 5:
        return 1
    else:
        return 2
    
def Title(x):
    if x == 'Mlle':
        return 'Miss'
    elif x == 'Ms':
        return 'Miss'
    elif x == 'Mme':
        return 'Mrs'
    elif x == 'Lady':
        return 'Miss'
    elif x  == 'Dona':
        return 'Miss'
    elif x  == 'Capt':
        return 'Officer'
    elif x == 'Col':
        return 'Officer'
    elif x == 'Major':
        return 'Officer'
    elif x == 'Dr':
        return 'Officer'
    elif x == 'Rev':
        return 'Officer'
    elif x == 'Don':
        return 'Officer'
    elif x == 'Sir':
        return 'Officer'
    elif x == 'the Countess':
        return 'Officer'
    elif x == 'Jonkheer':
        return 'Officer'
    else:
        return x
    
def TitleGroup(x):
    if x == 'Mr':
        return 0
    elif x == 'Miss':
        return 1
    elif x == 'Mrs':
        return 2
    elif x == 'Master':
        return 3
    elif x == 'Officer':
        return 4
    else:
        return 5
    
def TicketSize(x):
    if x == 1:
        return 0
    elif x < 5:
        return 1
    else:
        return 2
        

# comma delimited is the default
passengers = pd.read_csv(input_file, header = 0)

passengers['Sex'] = passengers \
    .apply(lambda row: row['Sex'] == 'female', axis = 1).astype(int)

passengers['Age'] = passengers['Age'].fillna(round(passengers['Age'].mean(), 2))

passengers['AgeGroup'] = passengers['Age'].apply(lambda x: AgeGroup(x))

passengers['Embarked'] = passengers['Embarked'] \
    .fillna(passengers['Embarked'].value_counts().idxmax())
    
passengers['Embarked'] = passengers['Embarked'] \
    .apply(lambda x: Embark(x)).astype(int)

passengers['Title'] = passengers['Name'] \
    .replace("^.*, (.*?)\\..*$", "\\1", regex = True) \
    .apply(lambda x: Title(x))

passengers['FamilySize'] = passengers[['SibSp', 'Parch']] \
    .apply(lambda x: FamilySize(x['SibSp'] + x['Parch'] + 1), axis = 1)
    
passengers['TitleGroup'] = passengers['Title'].apply(lambda x: TitleGroup(x))

passengers['TicketSize'] = passengers.groupby(['Ticket'])['Ticket'].transform('count')
passengers['TicketSize'] = passengers['TicketSize'].apply(lambda x: TicketSize(x))

pd.DataFrame.head(passengers, 5)

## Stochastic Gradient Descent

### Load train file into a Panda DataFrame

In [ ]:
import numpy as np
import pandas as pd

input_file = "./Sample/train.csv"
mnist = pd.read_csv(input_file, header = 0)

pd.DataFrame.head(mnist, 10)

### Separate mnist into two variables

In [ ]:
X, y = mnist.iloc[0:, 1:], mnist.iloc[0:,0]

print(X.shape)
print(y.shape)

### Print an image of the first row in mnist

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

some_digit = X.iloc[0,:]
some_digit_image = some_digit.values.reshape(28, 28)

plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation="nearest")
plt.axis("off")
plt.show()

### Split the mnist training data into a training and testing set

In [ ]:
X_train, X_test, y_train, y_test = X.iloc[:32000, :], X.iloc[32000:, :], y.iloc[:32000], y.iloc[32000:]

### Change the lables to true/false when the label equals five

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

### Create the SGD Classifier and fit/train it

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state = 42)
sgd_clf.fit(X_train, y_train_5)

### Print an image of a row in the test set that is a five

In [ ]:
some_digit = X_test.iloc[17,:]
some_digit_image = some_digit.values.reshape(28, 28)

plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation="nearest")
plt.axis("off")
plt.show()

### Check to see if SGD predicts five for a row in the test that is a five

In [ ]:
sgd_clf.predict([X_test.iloc[17,:]])

### Retrain SGD with full train set

In [ ]:
sgd_clf.fit(X, y)

### Load test file into a Panda DataFrame

In [ ]:
test_input_file = "./Sample/test.csv"
test_mnist = pd.read_csv(test_input_file, header = 0)

pd.DataFrame.head(test_mnist, 10)

### Predict the the values for the test set and save the results

In [ ]:
results = sgd_clf.predict(test_mnist)
results

### Create an image of the first row of the test set to verify the SGD predicted correctly

In [ ]:
some_digit = test_mnist.iloc[0,:]
some_digit_image = some_digit.values.reshape(28, 28)

plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation="nearest")
plt.axis("off")
plt.show()

### Export results to CSV to be uploaded to Kaggle

In [ ]:
row = range(1, results.size + 1)
result_df = pd.DataFrame({
    'ImageId': row,
    'Label': results
})

result_df[['ImageId', 'Label']].to_csv('./sgd_results.csv', sep=',', index = False, encoding='utf-8')

In [ ]:
f